### CS/ECE/ISyE 524 &mdash; Introduction to Optimization &mdash; Summer 2019 ###

# Supply Chain Optimization #

####   Xinping Liu (xliu735@wisc.edu);
####   Te Zhang (tzhang399@wisc.edu);
####   Siqi Du (sdu32@wisc.edu);
####   Yousef Shaheen (Yshaheen@wisc.edu);


### Table of Contents

1. [Introduction](#1.-Introduction)
1. [Two Stages](#2.-Two-Stages)
    1. [Assembly Line Optimization](#A.-Stage-1:-Assembly-Line-Optimization )
        1. [Mathematical Model](#a.-Mathematical-Model-for-Stage-1)
        1. [Solution](#b.-Solution-for-Stage-1)
            1. [Solution 1](#b.1-Solution-1-for-Stage-1)
            1. [Solution 2](#b.2-Solution-2-for-Stage-1)
        1. [Results and Discussion](#c.-Results-and-Discussion-for-Stage-1)
    1. [Transportation Optimization](#B.-Stage-2:-Transportation-Optimization)
        1. [Mathematical Model](#a.-Mathematical-Model-for-Stage-2)
             1. [Model 1](#a.1-MIP-Model-that-yields-the-minimum-number-of-warehouse)
             2. [Model 2](#a.2-MCNF-Transportation-Model-that-minimizes-the-shipping-cost)
        1. [Solution](#b.-Solution-for-Stage-2)
             1. [Solution 1](#b.1-MIP-Solution)
             2. [Solution 2](#b.2-MCNF-Trnasportation-Solution)
        1. [Results and Discussion](#c.-Results-and-Discussion-for-Stage-2)
1. [Conclusion](#3.-Conclusion)

# 1. Introduction #

A supply chain is a network between a company and its suppliers which together aim to produce and distribute a product or goods. This network includes different activities, people, entities, data, and resources [1]. A supply chain company is involoved in many steps along its way to deliver the final product, including but not limited to receiving orders from customers, developing and creating the product, marketing, shipping and distribution, and other managment tasks like accounting, finance, customer support, etc. Thus, A supply chain company tries to integrate these processes in the most efficient way, in order to reduce the cost of manufacturing and stay competitive in a global market. 

DNEE (Does Not Exist Electronics) a supply chain company, decided to go in the market for manufacturing and distributing the most sought after electronic products, due to increasing demand for electronic products around the world. As a result, DNEE hires a team of engineers to come up with an optimization model to help the company minimize the cost of both manufacturing the product and distributing it around the world. 

DNEE tells its engineers that the company would like to optimize the two major processes it is involved in. The first part starts at the manufacturer, and tha goal is to reduce the cost of manufacturing by figuring out the optimal duration of making the product. Once the product is ready to be shipped, the second optimization model kicks in. That is to reduce the cost of delivering the products from the company's warehouses to each of the distribution centers. It is worth noting that the second stage will not deal with shipping the product to its final destination, i.e. shipping the product from the distribution center to the final customer. Instead, it will tackle the problem of figuring out how many and how big the warehouses needs to be, to reduce the overall cost of delivering products to the distributing centers while meeting their customer demand from all around the world. 

In our model for a supply chain, we assume that DNEE has access to specific parts needed for manufacturing the final product. Each of the parts has a cost that is included in the making of that step. We assume that the number of distribution centers that is given can not be changed, but we may change the number of warehouses as long as we meet the demand of all of the distribution centers. We also assume that the demand data is given to us based on DNEE’s projected sales of a single product, and that various electronic products will be of the same or similar demand. 


Reference[1]: https://www.investopedia.com/terms/s/supplychain.asp


# 2. Two Stages #

## A. Stage 1: Assembly Line Optimization ##

The first stage is to help DNEE to figure out how to optimize the efficiency of the assembly line so that the total cost during the assembly process can be minimized.

Before the optimization, we know the assembly tasks, each task's duration, and their precedence relationship. Additionally, due to the technical limitations, each task has a minimum duration which is possible to be realized.

There are two types of cost that DNEE should take into account. One is the time cost. For each product, if the total assembly time is x minutes, the time cost will be x units. The other cost is the improvement cost, which is the money spent on reducing the tasks' durations. To reduce the duration of a task by t minutes, DNEE has to pay t^2 units cost.

We solve this problem by trading off between minimizing the time cost and the improvement cost. There may be more than one practical way to solve it, and we come up with two solutions. 

+ In the first solution, we firstly caculate the current critical path of the assmbly line, and then only consider reducing the durations of tasks on the critical path.
+ In the second solution, we consider reducing the durations of all tasks as long as possible.

Before implementing the solution, we assume that the two solutions will yield the same results.

## a. Mathematical Model for Stage 1
### 1. Data known for modeling
To improve the efficiency of the assembly line, we should know

+ The set of assembly tasks 
  $$ T = \left\{ Start, 1, 2, 3...Final\right\} $$  where Start represents the starting node, and Final represents the final node.
 
+ The current duration required for each assembly task
  $$ d_{i}\geq 0 \quad\forall i \in T $$  

+ The minimum duration possible for each assembly task
  $$ d_{i} \geq p_{i}\geq 0 \quad\forall i \in T $$
  
+ The tradeoff parameter which represents the weight between minimzing time cost and minimizing improvement cost.
  We randomly set the tradeoff parameter $$\lambda = 0.25 $$

+ The cost to reduce the duration: to reduce the duration of a task by t minutes, the improvement cost should equal to t^2 units
  
+ The precedence relationship of assembly tasks: shown in the table below

Table below is an example of data we need. Because the real data is difficult to collect, we randomly generate this example data based on the reality.  


| Steps |  Description          | Duration (min)     |  Predecessors    |  Minimum Duration Possible |     
|:-----:|:---------------------:| :-----------------:|:----------------:|:--------------------------:|
| A     | Mobo Installation     | 24                 | none             |  20
| B     | Chip Installation     | 15                 | A                |  12
| C     | CPU Installation      | 8                  | B                |  5
| D     | Memory Installation   | 12                 | B                |  8   
| E     | Cooler Installation   | 6                  | A                |  3
| F     | Storage Installation  | 12                 | A, E             |  7
| G     | Power Supply          | 9                  | B, F             |  5
| H     | Cable Installation    | 10                 | G                |  6
| I     | Screen Installation   | 5                  | G                |  4
| J     | Button Installation   | 4                  | H                |  4
| K     | Graphics and Sound    | 12                 | B, C, D          |  7
| L     | Power/Safety Test     | 10                 | J, K             |  8
| M     | Software Installation | 21                 | I                |  17
| N     | Tidying Up            | 8                  | I, J             |  7        


The graph below shows the current network flow, with corresponding duration for each task
![graph1](images/stage1-1.jpg)

### 2. Variable Definition
We need to find the current critical path, and then reduce the duration of tasks on that path. 
+ We define a variable $x_{i} \forall i \in S$, which represents the time we start working on task i when assembling PC.
+ We define a variable $n_{i} $ if task $i$ is on the critical path, which represents the new duration of each task after the improvement.
+ We define a variable $t_{i} $ if task $i$ is on the critical path, which represents the time reduced for each task. The cost needed for each task is equal to $t_{i}^2$.


### 3. Objective Function

+ Objective: Minimize Total Cost.

$$\begin{align*}
\underset{x}{\min} \  x_{Final} +  \lambda * \sum_{i=Start}^{Final} t_{i}^{2} \quad \forall i \in T
\end{align*}$$

+ Total Cost = Time Cost +  λ * Improvement Cost

$$\text{Time Cost} = x_{Final}$$

$$\text{Improvement Cost} = \sum_{i=Start}^{Final} t_{i}^{2} \quad \forall i \in T $$
 

### 4. Constraints

+ For every pair of task(i, j), if task j is a predessor of task i, then task i must start after task j 
    + Before the improvement, the duration for each task is $d_{i}$
       $$x_i \geq x_j + d_j \quad \forall i,j \in T $$
    + After the improvement, the duration for each task is $n_{i}$
       $$x_i \geq x_j + n_j \quad \forall i,j \in T $$

+ For each task, the current duration $d$ minus the time reduced $t$ should be equal to the new duration $n$ 
    $$ d_{i} - t_{i} = n_{i}  \quad \forall i \in T $$
+ For each task, the new duration $n$ cannot be less than its minimum duration possible $p$
    $$ p_{i} \leq n_{i} \quad \forall i \in T $$


### 5. Full Math Model

$$\begin{aligned}
\underset{x}{\min} \  x_{Final} +  \lambda * \sum_{i=Start}^{Final} t_{i}^{2}  \\
\end{aligned}$$

$$\begin{aligned}
\text{subject to}\quad x_i \geq x_j + d_j \\
 x_i \geq x_j + n_j \\
 d_{i} - t_{i} = n_{i}\\
 n_{i} \geq p_{i} \\
 d_{i}\geq 0 \\
 d_{i}\geq p_{i} \\
 p_{i}\geq 0 \\
 \lambda = 0.25\\
\end{aligned}$$

## b. Solution for Stage 1
### Data Definition
Before solving the problem, we define the data we need to use in order to use it in the solution.

In [15]:
using NamedArrays

# all the steps needed for manufacturing a PC
T = [:Start,:A, :B, :C, :D, :E, :F, :G, :H, :I, :J, :K, :L, :M, :N, :Final]

# the current duration for each task 
cur_duration = Dict(zip(T, [0, 24, 15, 8, 12, 6, 12, 9, 10, 5, 4, 12, 10, 21, 8, 0]));

# the minimum duration possible for each task
min_duration = Dict(zip(T, [0, 20, 12, 5, 8, 3, 7, 5, 6, 4, 4, 7, 8, 17, 7, 0]))

# The table is the representation of which steps preced others. Initially all elements are 0.
zero_list = zeros(16, 16)
pred_list = NamedArray(zero_list, (T, T),("Tasks", "Tasks") )

# if step j preceds step j, then assign value 1 to (j, i)
pred_list[:B, :A] = 1; pred_list[:C, :B] = 1; pred_list[:D, :B] = 1;
pred_list[:E, :A] = 1; pred_list[:F, :A] = 1; pred_list[:F, :E] = 1;
pred_list[:G, :B] = 1; pred_list[:G, :F] = 1; pred_list[:H, :G] = 1; 
pred_list[:I, :G] = 1; pred_list[:J, :H] = 1; pred_list[:K, :B] = 1;
pred_list[:K, :C] = 1; pred_list[:K, :D] = 1; pred_list[:L, :J] = 1;
pred_list[:L, :K] = 1; pred_list[:M, :I] = 1; pred_list[:N, :I] = 1;
pred_list[:N, :J] = 1; pred_list[:Final, :L] = 1;
pred_list[:Final, :M] = 1; pred_list[:Final, :N] = 1;

### b.1 Solution 1 for Stage 1
In the first solution, we firstly calculate the current critical path of the assembly line, and then only consider reducing the durations of tasks on the critical path. This is because only chaning the durations on the critical path will hava an effect on the total time of assembling. After that, we obtain the time cost by caculating the total duration of the assembling process, and obtain the improvement cost by caculating how much time has been reduced for each task. Then we can get the optimal objective using the tradeoff parameter λ. 

### Calculate the Current Critical Path

In [16]:
using JuMP, Clp, NamedArrays

m = Model(solver = ClpSolver())

@variable(m, x[T] >= 0) # time to finish each task

# create constraint for every pair of step(i,j) if step i is preceded by step j
@constraint(m,constr[i in T, j in T; pred_list[i,j] == 1], x[i] >= x[j] + cur_duration[j])

# minimize the time we start task Final (finish project)
@objective(m, Min, x[:Final])

solve(m)
solution = getvalue(x)

println()
println("The current critical path is: ")
# obtain the critical path of manufacturing PC
for i in T
    for j in T
        if pred_list[i,j] == 1
            if getdual(constr[i,j]) > 0
                # then we know this pair of tasks is on the critical path. 
                println("Task ", j)
            end
        end
    end
end


The current critical path is: 
Task A
Task E
Task F
Task G
Task I
Task M


###  Reducing the Durations of Tasks on the Critical Path & Get the Improvement Cost

We have got the current critical path, so we only try to improve the steps on the critical path. These Steps must be implemented in sequence, so there is no predecessors. 


In [17]:
using JuMP, Gurobi

# the steps on the critical path
CP = [:A, :E, :F, :G, :I, :M]

# the current duration for each step
cur_duration_CP = Dict(zip(CP, [24, 6, 12, 9, 5, 21]))

# the minimum duration possible for each step
min_duration_CP = Dict(zip(CP, [20, 3, 7, 5, 4, 17]))

# the tradeoff parameter
λ = 0.25

m = Model(solver = GurobiSolver())

@variable(m, new_duration[CP], Int) # the new duration for each step after improvement
@variable(m, t[CP] >= 0) # time reduced for each step

for i in CP
  # the current duration minus the reduced time should be equal to the new duration
  @constraint(m, cur_duration_CP[i] - t[i] == new_duration[i]) 
  # the new duration should not be less than the minimum possible duration  
  @constraint(m, new_duration[i] >= min_duration_CP[i])
end

# trade off between minimizing the time cost and minimizing the improvement cost
@objective(m, Min, sum(new_duration) + λ*sum(t[i]^2 for i in CP))

solve(m)

println()
for i in CP
    println("The duration of Task ", i, " is reduced from ", cur_duration_CP[i], 
        " minutes to ", getvalue(new_duration[i]), " minutes.")
end

println()
println("The improvement cost is: ", sum(getvalue(t[i])^2 for i in CP), " units." )

Academic license - for non-commercial use only
Optimize a model with 12 rows, 12 columns and 18 nonzeros
Model has 6 quadratic objective terms
Variable types: 6 continuous, 6 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  QObjective range [5e-01, 5e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 2e+01]
Presolve removed 6 rows and 0 columns
Presolve time: 0.00s
Presolved: 6 rows, 12 columns, 12 nonzeros
Presolved model has 6 quadratic objective terms
Variable types: 0 continuous, 12 integer (1 binary)
Found heuristic solution: objective 73.0000000

Root relaxation: objective 7.125000e+01, 18 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      71.2500000   71.25000  0.00%     -    0s

Explored 0 nodes (18 simplex iterations) in 0.00 seconds
Thread c

### Get the Time Cost & Get the Optimal Objective

Since we have already known how much time we should reduce for each task on the critical path, we should calculate the total duration of the assembling process after improvement, thus to obtain the time cost. Since we can also know the improvement cost, we can get the optimal objective using the tradeoff parameter λ.


In [18]:
using JuMP, Clp, NamedArrays

# the new duration each task after improvement
new_duration1 = Dict(zip(T, [0, 22, 15, 8, 12, 4, 10, 7, 10, 4, 4, 12, 10, 19, 8, 0]));

# the tradeoff parameter
λ = 0.25

m = Model(solver = ClpSolver())

@variable(m, x[T] >= 0) # time to finish each task

# create constraint for every pair of step(i,j) if step i is preceded by step j
@constraint(m,constr[i in T, j in T; pred_list[i,j] == 1], x[i] >= x[j] + new_duration1[j])

# minimize the time we start task Final (finish project)
@objective(m, Min, x[:Final])

solve(m)
solution = getvalue(x)

timeCost = getobjectivevalue(m)

improCost = 21 # we have got the improvement cost in the  

optimalObj = timeCost + λ*improCost

println("The time cost is: ", timeCost, " units.")
println("The improvement cost is: ", improCost, " units.") 
println("The optimal objective is: ", optimalObj)


The time cost is: 71.0 units.
The improvement cost is: 21 units.
The optimal objective is: 76.25


### b.2 Solution 2 for Stage 1
In the second solution, we consider reducing the durations of all tasks as long as possible. After the improvement, we calculate the new critical path, thus get the total time of the assembling process, and then obtain the time cost. Since we can also know the improvement cost, we can get the optimal objective using the tradeoff parameter λ.

###  Reducing the Durations of All the Tasks & Get the Optimal Objective 

In [19]:
using JuMP, Gurobi, Clp, NamedArrays

# the tradeoff parameter
λ = 0.25

m = Model(solver = GurobiSolver())

@variable(m, new_duration[T], Int) # the duration of each task after improvement
@variable(m, t[T] >= 0) # time reduced for each task
@variable(m, x[T] >= 0) # time to finish each task

# create constraint for every pair of step(i,j) if step i is preceded by step j
@constraint(m, constr[i in T, j in T; pred_list[i,j] == 1], x[i] >= x[j] + new_duration[j])
for i in T
  # the current duration minus the reduced time should be equal to the new duration
  @constraint(m, cur_duration[i] - t[i] == new_duration[i]) 
  # the new duration should not be less than the minimum possible duration  
  @constraint(m, new_duration[i] >= min_duration[i])
end

# trade off between minimizing the time cost and minimizing the improvement cost
@objective(m, Min, x[:Final] + λ*sum(t[i]^2 for i in T))

solve(m)
solution = getvalue(x)

println()
for i in T
    println("The duration of Task ", i, " is reduced from ", cur_duration[i], 
        " minutes to ", getvalue(new_duration[i]), " minutes.")
end

println()
println("The time cost is: ", getvalue(x[:Final]), " units")
println("The improvement cost is: ", getvalue(sum(t[i]^2 for i in T)) , " units")
println("The optimal objective value is: ", getobjectivevalue(m))

Academic license - for non-commercial use only
Optimize a model with 54 rows, 48 columns and 114 nonzeros
Model has 16 quadratic objective terms
Variable types: 32 continuous, 16 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  QObjective range [5e-01, 5e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 2e+01]
Found heuristic solution: objective 2.000000e+09
Presolve removed 36 rows and 20 columns
Presolve time: 0.00s
Presolved: 18 rows, 28 columns, 49 nonzeros
Presolved model has 13 quadratic objective terms
Found heuristic solution: objective 117.7500000
Variable types: 2 continuous, 26 integer (2 binary)

Root relaxation: objective 7.203125e+01, 46 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

H    0     0                     101.5000000   58.00000  42.9%     -    0s
     0  

## c. Results and Discussion for Stage 1 

Different from our initial assumption, the two solutions get different optimal objective values, along with different improvement plans. 

It is probably because that in solution 1, the critical path was changed as the task durations changed. So only improving tasks on the previous path is not the most efficient solution.

To verify our hypothesis, we re-calculate the critical path after making improvement in solution 1, to see whether the critical path has changed. If the path changed, it would suggest that the first solution is not efficient enough at solving this problem.

For solution 1, the new duration for each task after improvement should be as follows:


| Steps |  Description          | New Duration (min) |  Predecessors    |    
|:-----:|:---------------------:| :-----------------:|:----------------:|
| A     | Mobo Installation     | 22                 | none             |  
| B     | Chip Installation     | 15                 | A                | 
| C     | CPU Installation      | 8                  | B                |  
| D     | Memory Installation   | 12                 | B                |     
| E     | Cooler Installation   | 4                  | A                |  
| F     | Storage Installation  | 10                 | A, E             |  
| G     | Power Supply          | 7                  | B, F             |  
| H     | Cable Installation    | 10                 | G                |  
| I     | Screen Installation   | 4                  | G                |  
| J     | Button Installation   | 4                  | H                |  
| K     | Graphics and Sound    | 12                 | B, C, D          |  
| L     | Power/Safety Test     | 10                 | J, K             |  
| M     | Software Installation | 19                 | I                |  
| N     | Tidying Up            | 8                  | I, J             |  


In [20]:
using JuMP, Clp, NamedArrays

# each step has required time (duration) to be completed
new_duration1 = Dict(zip(T, [0, 22, 15, 8, 12, 4, 10, 7, 10, 4, 4, 12, 10, 19, 8, 0]));

m = Model(solver = ClpSolver())

@variable(m, x[T] >= 0) # time to finish each task

# create constraint for every pair of step(i,j) if step i is preceded by step j
@constraint(m,constr[i in T, j in T; pred_list[i,j] == 1], x[i] >= x[j] + new_duration1[j])

# minimize the time we start task Final (finish project)
@objective(m, Min, x[:Final])

solve(m)
solution = getvalue(x)

println("The minimum manufacturing time required for PC is : ", getobjectivevalue(m), " minutes")

println()
println("The critical path is: ")
# obtain the critical path of manufacturing PC
for i in T
    for j in T
        if pred_list[i,j] == 1
            if getdual(constr[i,j]) > 0
                # then we know this pair of tasks is on the critical path. 
                println("Task ", j)
            end
        end
    end
end

The minimum manufacturing time required for PC is : 71.0 minutes

The critical path is: 
Task A
Task B
Task D
Task K
Task L


We can clearly see that the critical path has change after making improvement in solution 1. It suggests that solution is not efficient enough to get the most optimal objective.

For solution 2, the new duration for each task after improvement should be as follows:


| Steps |  Description          | New Duration (min) |  Predecessors    |    
|:-----:|:---------------------:| :-----------------:|:----------------:|
| A     | Mobo Installation     | 22                 | none             |  
| B     | Chip Installation     | 15                 | A                | 
| C     | CPU Installation      | 8                  | B                |  
| D     | Memory Installation   | 12                 | B                |     
| E     | Cooler Installation   | 4                  | A                |  
| F     | Storage Installation  | 11                 | A, E             |  
| G     | Power Supply          | 8                  | B, F             |  
| H     | Cable Installation    | 10                 | G                |  
| I     | Screen Installation   | 4                  | G                |  
| J     | Button Installation   | 4                  | H                |  
| K     | Graphics and Sound    | 11                 | B, C, D          |  
| L     | Power/Safety Test     | 9                  | J, K             |  
| M     | Software Installation | 20                 | I                |  
| N     | Tidying Up            | 8                  | I, J             |  

This is the most optimal improvement plan for DNEE.

We can then compute the critical path to determine the sequence when implementing the tasks.

In [21]:
using JuMP, Clp, NamedArrays

# each step has required time (duration) to be completed
new_duration2 = Dict(zip(T, [0, 22, 15, 8, 12, 4, 11, 8, 10, 4, 4, 11, 9, 20, 8, 0]));

m = Model(solver = GurobiSolver())

@variable(m, x[T] >= 0) # time to finish each task

# create constraint for every pair of step(i,j) if step i is preceded by step j
@constraint(m,constr[i in T, j in T; pred_list[i,j] == 1], x[i] >= x[j] + new_duration2[j])

# minimize the time we start task Final (finish project)
@objective(m, Min, x[:Final])

solve(m)
solution = getvalue(x)

println("The minimum manufacturing time required for PC is : ", getobjectivevalue(m), " minutes")

println()
println("The critical path is: ")
# obtain the critical path of manufacturing PC
for i in T
    for j in T
        if pred_list[i,j] == 1
            if getdual(constr[i,j]) > 0
                # then we know this pair of tasks is on the critical path. 
                println("Task ", j)
            end
        end
    end
end

Academic license - for non-commercial use only
Optimize a model with 22 rows, 16 columns and 44 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 2e+01]
Presolve removed 22 rows and 16 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.9000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  6.900000000e+01
The minimum manufacturing time required for PC is : 69.0 minutes

The critical path is: 
Task A
Task B
Task G
Task I
Task M


The graph below shows the network flow after the improvement along with the critical path.
![graph2](images/stage1-2.jpg)

# B. Stage 2: Transportation Optimization
The second stage is composed of two parts. The first part is to minimize the number of warehouses that we need to build, while meeting the demand of distribution centers around the world. We first start with an Integer Program, which will yeild to us the minimum number of warehouse to build. Once we have the right amount of warhouses, the second part would be to minimize the shipping cost to all distribution centers, while still meeting thier demand. This will be a Min-Cost Network Flow problem, specifically a transportation problem.

## a. Mathematical Model for Stage 2
## a.1 MIP Model that yields the minimum number of warehouse

In this part, a Mixed Integer Program is used to minimize the number of warehouses. This MIP will be based on the set covering problem and the following assumptions:
+ To make it real, the distance between two cities is reduced by scale
+ We want to make sure that all distribution center will be covered by at least one warehouse
+ At least one warehouse is within 20 units distance to each distribution center




The data given is listed here:  

| Distribution Center/Warehouse |  Chicago  |   Rome   |  Hong Kong  |  Lusaka  |  Moscow  | New Delhi |
|:-----------------------------:|:---------:|:--------:|:-----------:|:--------:|:--------:|:---------:|
| Boston air                    |10         |25        |40           |35        |70        |80
| Berlin air                    |25         |15        |30           |20        |30        |30
| Cairo air                     |30         |20        |25           |15        |20        |20
| Tokyo air                     |40         |25        |10           |20        |20        |20
| Ottawa air                    |15         |25        |60           |35        |70        |85
| Melbourne air                 |50         |40        |17           |30        |60        |30
| Rio de Janeiro air            |20         |40        |60           |30        |80        |50

### 1. Data and variables definition

+ We define $w$ the set of warehouses
+ We define $dc$ the set of distribution centers
+ We define a variable $x_{j}  \forall j \in w$, which represents the number of orders we send in each arcs
+ We define $a_{i,j}$ the entries of matrix A

### 2. Constraints

+ Make sure that all distribution centers are covered by at least one warehouse  
$$\begin{aligned} \sum_{i=1}^{7}a_{ij}*x_i \geq 1 && j=1,\dots,6 \end{aligned}$$

+ $x_j$ should be binary variables
$$\begin{aligned} x_j \in \{0,1\} && j=1,\dots,6 \end{aligned}$$

### 3. Objective
+ Minimize the number of warehouses
$$\begin{aligned}
  \underset{x}{\text{Min}}\qquad& \sum_{j=1}^{6}x_j \\
    \end{aligned}$$

### 4. Full Math Model
$$\begin{aligned}
  \underset{x}{\text{Min}}\qquad& \sum_{j=1}^{6}x_j \\
    \text{s.t.}\qquad& \sum_{i=1}^{7}a_{ij}*x_i \geq 1 && j=1,\dots,6\\
    & x_j \in \{0,1\} && j=1,\dots,6
    \end{aligned}$$

## a.2 MCNF Transportation Model that minimizes the shipping cost
In this part, we are using a MCNF LP, specifically the transportation model, to minimize the cost of our shipping cost to between the different warehouses and distribution centers around the world. The following assumptions are made:

+ Costs of rail and air transport per order are calculated based on the distance of two cities.

+ Because of travel distance restrictions, some routes, which don't have the cost, are infeasible.

+ Each warehouse has its own capacity and the total number of orders one warehouse delivers should not exceed its capacity.

+ Each distribution center has its own demand requested by the local customers.

+ We still need to meet the lower bound limit on air transport to three biggest cities.

The data given is listed here:   

| Distribution Center/Warehouse |  Chicago  |   Rome   |  Hong Kong  |  Lusaka  |  Distribution center demand  |
|:-----------------------------:|:---------:|:--------:|:-----------:|:--------:|:----------------------------:|
| Boston air(rail)              |5k (3k)    |10k (-)   |12k (-)      |8k (-)    |80
| Berlin air(rail)              |10k (-)    |6k (4k)   |8k (6k)      |7k (-)    |90
| Cairo air(rail)               |- (-)      |5k (-)    |6k (-)       |6k (2k)   |30
| Tokyo air(rail)               |12k (-)    |8k (-)    |4k (2k)      |8k (-)    |40
| Ottawa air(rail)              |7k (5k)    |9k (-)    |- (-)        |10k (-)   |70
| Melbourne air(rail)           |- (-)      |13k (-)   |6k (-)       |15k (-)   |35
| Rio de Janeiro air(rail)      |12k (-)    |15k (-)   |- (-)        |5k (-)    |40
| Warehouse capacity            |100        |100       |100          |100       |

|Biggest distribution center    | Boston | Berlin | Ottawa |
|:-----------------------------:|:------:|:------:|:------:|
|minimum number of orders by air|20      |25      |15      |

The graph visualizes our current transportation paths. Due to the complexity of our data, we just draw the paths from one warehouse to all distribution centers
![graph3](images/stage2.jpg)

### 1. Data and variables definition
+ We define $w$ the set of warehouses we choose to build
+ We define $dc$ the set of distribution centers
+ We define $nodes$ as the set of nodes in the network flow graph, which is indexed by either air or rail
+ We define $arcs$ as the set of arcs that exist between different warhouse and distribution centers, as well as the dummy nodes s and t (source and sink)
+ We define $c$ as the cost of each of the routes that exist between a warehouse and a distribution center (including both air and rail)
+ We define $max_cap$ as the maximum capacity each route can take (for both air and rail)
+ We define a variable $x_{i}  \forall i \in arcs$, which represents the number of orders we send on each arcs, i.e. each route


### 2. Constraints

+ Balance flow constraint. For all nodes, except sourse s and sink t, number of orders into each node equals to number of orders out of that node
$$\begin{aligned} \qquad& \displaystyle \sum_{j \in \mathbb{N}}^{}x_k,_j = \displaystyle \sum_{i \in \mathbb{N}}^{}x_i,_k\,\, , \forall k \in N = nodes -\{s,t\} \end{aligned}$$

+ Amount of orders sent on each route does not exceed the maximum capacity 
$$\begin{aligned} x_i \le cap_i\,\,, \forall i \in arcs \end{aligned}$$


+ Meet the demand of each distribution centere, sum of orders sent using air and rail = distribution center demand
$$\begin{aligned} \qquad&x_{BsA,t} + x_{BsR,t} = 80 \\ \qquad&x_{BrA,t} + x_{BrR,t} = 90 \\ \qquad&x_{CrA,t} + x_{CrR,t} = 30 \\ \qquad&x_{TA,t} + x_{TR,t} = 40 \\ \qquad&x_{OA,t} + x_{OR,t} = 80 \\ \qquad&x_{MA,t} + x_{MR,t} = 35 \\ \qquad&x_{RA,t} + x_{RR,t} = 40 \\ \end{aligned}$$


+ Make sure to meet airline minimum order requirements
$$\begin{aligned} \qquad&x_{BsA,t} \ge 20 \\ \qquad&x_{BrA,t}\ge25 \\ \qquad&x_{OA,t} \ge15 \end{aligned}$$

### 3. Objective

+ Minimize the sum of total cost of each of the valid routes
$$\begin{aligned}
  \underset{x}{\text{Min}}\qquad& \displaystyle \sum_{i \in \mathbb{arcs}}^{}
    x_i * c_i \end{aligned}$$

### 4. Full Math Model
$$\begin{aligned}
  \underset{x}{\text{Min}}\qquad& \displaystyle \sum_{i \in \mathbb{arcs}}^{}
    x_i * c_i \\\text{subject to:}\qquad& \displaystyle \sum_{j \in \mathbb{N}}^{}x_k,_j = \displaystyle \sum_{i \in \mathbb{N}}^{}x_i,_k\,\, , \forall k \in N = nodes -\{s,t\}\\ \qquad& x_i \le cap_i\,\,, \forall i \in arcs \\ \qquad&x_{BsA,t} + x_{BsR,t} = 80 \\ \qquad&x_{BrA,t} + x_{BrR,t} = 90 \\ \qquad&x_{CrA,t} + x_{CrR,t} = 30 \\ \qquad&x_{TA,t} + x_{TR,t} = 40 \\ \qquad&x_{OA,t} + x_{OR,t} = 80 \\ \qquad&x_{MA,t} + x_{MR,t} = 35 \\ \qquad&x_{RA,t} + x_{RR,t} = 40 \\
    \qquad&x_{BsA,t} \ge 20 \\ \qquad&x_{BrA,t}\ge25 \\ \qquad&x_{OA,t} \ge15
    \end{aligned}$$

## b. Solution for Stage 2
## b.1 MIP Solution


In [22]:
using JuMP, Gurobi, NamedArrays
w=[:C,:R,:H,:L, :M, :N] # warehouses
dc=[:Bs,:Br,:Cr,:T,:O,:M,:R]# distribution centers


dist = NamedArray([10 25 40 35 70 80
                   22 15 30 25 30 30
                   30 20 20 15 20 20
                   40 25 10 20 25 20
                   15 25 60 35 70 85
                   50 40 17 30 60 30
                   25 40 60 20 80 50],(dc,w),("distribution center","warehouse"))
println(dist)

A = [1 0 0 0 0 0
     0 1 0 0 0 0
     0 1 1 1 1 1
     0 0 1 1 0 1
     1 0 0 0 0 0
     0 0 1 0 0 0
     0 0 0 1 0 0] # the cut-off number is 20, the distance below 20 should be 1, and up 20 should be zero



7×6 Named Array{Int64,2}
distribution center ╲ warehouse │ :C  :R  :H  :L  :M  :N
────────────────────────────────┼───────────────────────
:Bs                             │ 10  25  40  35  70  80
:Br                             │ 22  15  30  25  30  30
:Cr                             │ 30  20  20  15  20  20
:T                              │ 40  25  10  20  25  20
:O                              │ 15  25  60  35  70  85
:M                              │ 50  40  17  30  60  30
:R                              │ 25  40  60  20  80  50


7×6 Array{Int64,2}:
 1  0  0  0  0  0
 0  1  0  0  0  0
 0  1  1  1  1  1
 0  0  1  1  0  1
 1  0  0  0  0  0
 0  0  1  0  0  0
 0  0  0  1  0  0

In [23]:
using Gurobi
m = Model(solver=GurobiSolver(OutputFlag=0))

@variable(m, x[1:6], Bin)

for i = 1:length(dc)
     # Make sure that all distribution centers are covered by at least one warehouse
    @constraint(m, sum(A[i,j]*x[j] for j = 1:length(w) ) >= 1)  
end

@objective(m, Min, sum(x))
                
solve(m)

println(getvalue(x))

Academic license - for non-commercial use only
[1.0, 1.0, 1.0, 1.0, 0.0, 0.0]


From the model and solution above, we see that four warehouses can cover all demands of distribution centers. This means that closing the warehouse in Moscow and New Delhi can help our company reduce the cost without compromising any customers.

## b.2 MCNF Trnasportation Solution

In [24]:
w=[:C,:R,:H,:L] # warehouses
dc=[:Bs,:Br,:Cr,:T,:O,:M,:R]# distribution centers

# nodes in the network. index destinations by both air and rail
nodes=[:s;w;:BsA;:BsR;:BrA;:BrR;:CrA;:CrR;:TA;:TR;:OA;:OR;:MA;:MR;:RA;:RR;:t]

# make a list of arcs (as tuples) that exist in the network
arcs=[(:s,:C),(:s,:R),(:s,:H),(:s,:L),(:C,:BsA),(:C,:BsR),(:C,:BrA),(:C,:TA),
      (:C,:OA),(:C,:OR),(:C,:RA),(:R,:BsA),(:R,:BrA),(:R,:BrR),(:R,:CrA),(:R,:TA),(:R,:OA),
      (:R,:MA),(:R,:RA),(:H,:BsA),(:H,:BrA),(:H,:BrR),(:H,:CrA),(:H,:TA),(:H,:TR),(:H,:MA),
      (:L,:BsA),(:L,:BrA),(:L,:CrA),(:L,:CrR),(:L,:TA),(:L,:OA),(:L,:MA),(:L,:RA),(:BsA,:t),
      (:BsR,:t),(:BrA,:t),(:BrR,:t),(:CrA,:t),(:CrR,:t),(:TA,:t),(:TR,:t),(:OA,:t),(:OR,:t),
      (:MA,:t),(:MR,:t),(:RA,:t),(:RR,:t)]

# dictionary of travel cost (per order) on each arc
c=Dict(zip(arcs,[0, 0, 0, 0, 5, 3, 10, 12, 7, 5, 12, 10, 6, 4, 5,
            8, 9, 13, 15, 12, 8, 6, 6, 4, 2, 6, 8, 7, 6, 2, 8, 10, 15,
            5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))


#max capacity of each arc
max_cap = Dict(zip(arcs,[100, 100, 100, 100, 45, 60, 45, 45, 45, 60, 45, 45, 45, 60, 45, 45, 
            45, 45, 45, 45, 45, 60, 45, 45, 60, 45, 45, 45, 45, 60, 45, 45, 45, 45, 500, 500,
            500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500]))

using JuMP, Clp
m = Model(solver=ClpSolver())

# variable for number of orders sent on each arc
@variable(m, x[arcs] >= 0)

        
# minimize the total cost of shipping orders
@objective(m, Min, sum(c[a]*x[a] for a in arcs))
        
# for each node
for i in nodes
    # if node is not source or sink
    if i != :s && i != :t
         # balance flow into and out of node
         @constraint(m, sum(x[j] for j in arcs if j[2] == i) == sum(x[j] for j
        in arcs if j[1] == i))
    end
end
                                
# don't exceed capacity restrictions on each arc
@constraint(m, cap[i in arcs], x[i] <= max_cap[i])
                             
# meet the demand of each of distribution center
@constraint(m, x[(:BsA,:t)] + x[(:BsR,:t)] == 80)
@constraint(m, x[(:BrA,:t)] + x[(:BrR,:t)] == 90)
@constraint(m, x[(:CrA,:t)] + x[(:CrR,:t)] == 30)
@constraint(m, x[(:TA,:t)] + x[(:TR,:t)] == 40)
@constraint(m, x[(:OA,:t)] + x[(:OR,:t)] == 70)
@constraint(m, x[(:MA,:t)] + x[(:MR,:t)] == 35)
@constraint(m, x[(:RA,:t)] + x[(:RR,:t)] == 40)
                                    
# meet airline requirments, to keep getting a low rate
@constraint(m, x[(:BsA,:t)] >= 20)
@constraint(m, x[(:BrA,:t)] >= 25)
@constraint(m, x[(:OA,:t)] >= 15)
                            


solve(m)

println("Shipping cost: \$", getobjectivevalue(m))
println("Number of orders received by sink (should be 385): ", sum(getvalue(x[i]) for i in arcs if i[2] == :t))
println("Ship orders as follows: ")
println(getvalue(x))
                                            
getdual(cap)

Shipping cost: $1800.0
Number of orders received by sink (should be 385): 385.0
Ship orders as follows: 
x: 1 dimensions:
[  (:s, :C)] = 100.0
[  (:s, :R)] = 100.0
[  (:s, :H)] = 85.0
[  (:s, :L)] = 100.0
[(:C, :BsA)] = 0.0
[(:C, :BsR)] = 60.0
[(:C, :BrA)] = 0.0
[ (:C, :TA)] = 0.0
[ (:C, :OA)] = 0.0
[ (:C, :OR)] = 40.0
[ (:C, :RA)] = 0.0
[(:R, :BsA)] = 0.0
[(:R, :BrA)] = 25.0
[(:R, :BrR)] = 55.0
[(:R, :CrA)] = 0.0
[ (:R, :TA)] = 0.0
[ (:R, :OA)] = 20.0
[ (:R, :MA)] = 0.0
[ (:R, :RA)] = 0.0
[(:H, :BsA)] = 0.0
[(:H, :BrA)] = 0.0
[(:H, :BrR)] = 10.0
[(:H, :CrA)] = 0.0
[ (:H, :TA)] = 0.0
[ (:H, :TR)] = 40.0
[ (:H, :MA)] = 35.0
[(:L, :BsA)] = 20.0
[(:L, :BrA)] = 0.0
[(:L, :CrA)] = 0.0
[(:L, :CrR)] = 30.0
[ (:L, :TA)] = 0.0
[ (:L, :OA)] = 10.0
[ (:L, :MA)] = 0.0
[ (:L, :RA)] = 40.0
[(:BsA, :t)] = 20.0
[(:BsR, :t)] = 60.0
[(:BrA, :t)] = 25.0
[(:BrR, :t)] = 65.0
[(:CrA, :t)] = 0.0
[(:CrR, :t)] = 30.0
[ (:TA, :t)] = 0.0
[ (:TR, :t)] = 40.0
[ (:OA, :t)] = 30.0
[ (:OR, :t)] = 40.0
[ (:MA, :t)] = 

__anon__: 1 dimensions:
[  (:s, :C)] = -6.0
[  (:s, :R)] = -2.0
[  (:s, :H)] = 0.0
[  (:s, :L)] = -1.0
[(:C, :BsA)] = 0.0
[(:C, :BsR)] = 0.0
[(:C, :BrA)] = 0.0
[ (:C, :TA)] = 0.0
[ (:C, :OA)] = 0.0
[ (:C, :OR)] = 0.0
[ (:C, :RA)] = 0.0
[(:R, :BsA)] = 0.0
[(:R, :BrA)] = 0.0
[(:R, :BrR)] = 0.0
[(:R, :CrA)] = 0.0
[ (:R, :TA)] = 0.0
[ (:R, :OA)] = 0.0
[ (:R, :MA)] = 0.0
[ (:R, :RA)] = 0.0
[(:H, :BsA)] = 0.0
[(:H, :BrA)] = 0.0
[(:H, :BrR)] = 0.0
[(:H, :CrA)] = 0.0
[ (:H, :TA)] = 0.0
[ (:H, :TR)] = 0.0
[ (:H, :MA)] = 0.0
[(:L, :BsA)] = 0.0
[(:L, :BrA)] = 0.0
[(:L, :CrA)] = 0.0
[(:L, :CrR)] = 0.0
[ (:L, :TA)] = 0.0
[ (:L, :OA)] = 0.0
[ (:L, :MA)] = 0.0
[ (:L, :RA)] = 0.0
[(:BsA, :t)] = 0.0
[(:BsR, :t)] = 0.0
[(:BrA, :t)] = 0.0
[(:BrR, :t)] = 0.0
[(:CrA, :t)] = 0.0
[(:CrR, :t)] = 0.0
[ (:TA, :t)] = 0.0
[ (:TR, :t)] = 0.0
[ (:OA, :t)] = 0.0
[ (:OR, :t)] = 0.0
[ (:MA, :t)] = 0.0
[ (:MR, :t)] = 0.0
[ (:RA, :t)] = 0.0
[ (:RR, :t)] = 0.0

## c. Results and Discussion for Stage 2
From the solution above, we see that the minimum cost will be equal to $1800. Each route has a specifice number of orders to send through, whether it is air or rail. However, we notice from the dual values of the  capaicty that some constraints has slackness and some are active. To be specific, the arcs we are interseted in are the following values: 
(:s, :C) = -6.0
(:s, :R) = -2.0
(:s, :H) = 0.0
(:s, :L) = -1.0

From here, we decide to increase the capacity of the warehouse that corresponds to the largest lambda value (:s, :C), which means that the chicago warhouse capcity should be increased. Also, we decided to reduce the capacity of the warehouse that has lackness (:s, :H), which means that Hong Kong warehouse should be reduced.

In [25]:
w=[:C,:R,:H,:L] # warehouses
dc=[:Bs,:Br,:Cr,:T,:O,:M,:R]# distribution centers
#modes=[:air,:rail]# possible modes of transportation


# nodes in the network. index destinations by both air and rail
nodes=[:s;w;:BsA;:BsR;:BrA;:BrR;:CrA;:CrR;:TA;:TR;:OA;:OR;:MA;:MR;:RA;:RR;:t]

# make a list of arcs (as tuples) that exist in the network
arcs=[(:s,:C),(:s,:R),(:s,:H),(:s,:L),(:C,:BsA),(:C,:BsR),(:C,:BrA),(:C,:TA),
      (:C,:OA),(:C,:OR),(:C,:RA),(:R,:BsA),(:R,:BrA),(:R,:BrR),(:R,:CrA),(:R,:TA),(:R,:OA),
      (:R,:MA),(:R,:RA),(:H,:BsA),(:H,:BrA),(:H,:BrR),(:H,:CrA),(:H,:TA),(:H,:TR),(:H,:MA),
      (:L,:BsA),(:L,:BrA),(:L,:CrA),(:L,:CrR),(:L,:TA),(:L,:OA),(:L,:MA),(:L,:RA),(:BsA,:t),
      (:BsR,:t),(:BrA,:t),(:BrR,:t),(:CrA,:t),(:CrR,:t),(:TA,:t),(:TR,:t),(:OA,:t),(:OR,:t),
      (:MA,:t),(:MR,:t),(:RA,:t),(:RR,:t)]

# dictionary of travel cost (per order) on each arc
c=Dict(zip(arcs,[0, 0, 0, 0, 5, 3, 10, 12, 7, 5, 12, 10, 6, 4, 5,
            8, 9, 13, 15, 12, 8, 6, 6, 4, 2, 6, 8, 7, 6, 2, 8, 10, 15,
            5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))


#max capacity of each arc
max_cap = Dict(zip(arcs,[150, 100, 80, 100, 45, 60, 45, 45, 45, 60, 45, 45, 45, 60, 45, 45, 
            45, 45, 45, 45, 45, 60, 45, 45, 60, 45, 45, 45, 45, 60, 45, 45, 45, 45, 500, 500,
            500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500]))

using JuMP, Clp
m = Model(solver=ClpSolver())

# variable for number of orders sent on each arc
@variable(m, x[arcs] >= 0)

        
# minimize the total cost of shipping orders
@objective(m, Min, sum(c[a]*x[a] for a in arcs))
        
# for each node
for i in nodes
    # if node is not source or sink
    if i != :s && i != :t
         # balance flow into and out of node
         @constraint(m, sum(x[j] for j in arcs if j[2] == i) == sum(x[j] for j
        in arcs if j[1] == i))
    end
end
                                
# don't exceed capacity restrictions on each arc
@constraint(m, cap[i in arcs], x[i] <= max_cap[i])
                             
# meet the demand of each of distribution center
@constraint(m, x[(:BsA,:t)] + x[(:BsR,:t)] == 80)
@constraint(m, x[(:BrA,:t)] + x[(:BrR,:t)] == 90)
@constraint(m, x[(:CrA,:t)] + x[(:CrR,:t)] == 30)
@constraint(m, x[(:TA,:t)] + x[(:TR,:t)] == 40)
@constraint(m, x[(:OA,:t)] + x[(:OR,:t)] == 70)
@constraint(m, x[(:MA,:t)] + x[(:MR,:t)] == 35)
@constraint(m, x[(:RA,:t)] + x[(:RR,:t)] == 40)
                                    
# meet airline requirments, to keep getting a low rate
@constraint(m, x[(:BsA,:t)] >= 20)
@constraint(m, x[(:BrA,:t)] >= 25)
@constraint(m, x[(:OA,:t)] >= 15)
                            


solve(m)

println("Shipping cost: \$", getobjectivevalue(m))
println("Number of orders received by sink (should be 385): ", sum(getvalue(x[i]) for i in arcs if i[2] == :t))
println("Ship orders as follows: ")
println(getvalue(x))
                                            
getdual(cap)

Shipping cost: $1630.0
Number of orders received by sink (should be 385): 385.0
Ship orders as follows: 
x: 1 dimensions:
[  (:s, :C)] = 150.0
[  (:s, :R)] = 85.0
[  (:s, :H)] = 80.0
[  (:s, :L)] = 70.0
[(:C, :BsA)] = 20.0
[(:C, :BsR)] = 60.0
[(:C, :BrA)] = 0.0
[ (:C, :TA)] = 0.0
[ (:C, :OA)] = 15.0
[ (:C, :OR)] = 55.0
[ (:C, :RA)] = 0.0
[(:R, :BsA)] = 0.0
[(:R, :BrA)] = 25.0
[(:R, :BrR)] = 60.0
[(:R, :CrA)] = 0.0
[ (:R, :TA)] = 0.0
[ (:R, :OA)] = 0.0
[ (:R, :MA)] = 0.0
[ (:R, :RA)] = 0.0
[(:H, :BsA)] = 0.0
[(:H, :BrA)] = 0.0
[(:H, :BrR)] = 5.0
[(:H, :CrA)] = 0.0
[ (:H, :TA)] = 0.0
[ (:H, :TR)] = 40.0
[ (:H, :MA)] = 35.0
[(:L, :BsA)] = 0.0
[(:L, :BrA)] = 0.0
[(:L, :CrA)] = 0.0
[(:L, :CrR)] = 30.0
[ (:L, :TA)] = 0.0
[ (:L, :OA)] = 0.0
[ (:L, :MA)] = 0.0
[ (:L, :RA)] = 40.0
[(:BsA, :t)] = 20.0
[(:BsR, :t)] = 60.0
[(:BrA, :t)] = 25.0
[(:BrR, :t)] = 65.0
[(:CrA, :t)] = 0.0
[(:CrR, :t)] = 30.0
[ (:TA, :t)] = 0.0
[ (:TR, :t)] = 40.0
[ (:OA, :t)] = 15.0
[ (:OR, :t)] = 55.0
[ (:MA, :t)] = 35.0

__anon__: 1 dimensions:
[  (:s, :C)] = 0.0
[  (:s, :R)] = 0.0
[  (:s, :H)] = 0.0
[  (:s, :L)] = 0.0
[(:C, :BsA)] = 0.0
[(:C, :BsR)] = 0.0
[(:C, :BrA)] = 0.0
[ (:C, :TA)] = 0.0
[ (:C, :OA)] = 0.0
[ (:C, :OR)] = 0.0
[ (:C, :RA)] = 0.0
[(:R, :BsA)] = 0.0
[(:R, :BrA)] = 0.0
[(:R, :BrR)] = -2.0
[(:R, :CrA)] = 0.0
[ (:R, :TA)] = 0.0
[ (:R, :OA)] = 0.0
[ (:R, :MA)] = 0.0
[ (:R, :RA)] = 0.0
[(:H, :BsA)] = 0.0
[(:H, :BrA)] = 0.0
[(:H, :BrR)] = 0.0
[(:H, :CrA)] = 0.0
[ (:H, :TA)] = 0.0
[ (:H, :TR)] = 0.0
[ (:H, :MA)] = 0.0
[(:L, :BsA)] = 0.0
[(:L, :BrA)] = 0.0
[(:L, :CrA)] = 0.0
[(:L, :CrR)] = 0.0
[ (:L, :TA)] = 0.0
[ (:L, :OA)] = 0.0
[ (:L, :MA)] = 0.0
[ (:L, :RA)] = 0.0
[(:BsA, :t)] = 0.0
[(:BsR, :t)] = 0.0
[(:BrA, :t)] = 0.0
[(:BrR, :t)] = 0.0
[(:CrA, :t)] = 0.0
[(:CrR, :t)] = 0.0
[ (:TA, :t)] = 0.0
[ (:TR, :t)] = 0.0
[ (:OA, :t)] = 0.0
[ (:OR, :t)] = 0.0
[ (:MA, :t)] = 0.0
[ (:MR, :t)] = 0.0
[ (:RA, :t)] = 0.0
[ (:RR, :t)] = 0.0

We notice that the total cost for shipping has gone down. This means that our company should expand the chicago location to accomedate for more orders. At the same time, Hongkong warehouse's capacity should be reduced.

From the models and solutions above, for Stage 2, we conclude that we only need four warehouses: Chicago, Rome, Hongkong, Lusaka to meet the demand of distribution cenetres around the world. We also found out how many orders to send per shipping route, including air and rail to meet the demand and minmize the cost. Finally, we concluded that increasing the capacity of the Chicago warehouse and reducing the capacity of the Hongkong warehouse will reduce the total cost of shipping even further.


## 3. Conclusion ##

In conclusion, by looking at the solutions for the above two stages, DNEE can reduce total cost of its supply chain model by following these recommendations: First, minimize the cost of each of the given manufacturing steps involved in making the product. Then, optimize the order of those steps by finding the critical path with the least cost, as show above in the second part of the first stage. Second, DNEE should invest in building only four warehouses around the world. And to meet the demand, it should addapt the suggested number of orders per route to transfer products between warehouses and distribution centers. Further, we found that increasing the capacity of some warehouses, like the one in Chicago and reducing others, like the one in Hongkong lowers cost of shipping even more. 

As mentioned earlier, delivering products to their final destination is not done by DNEE. Instead, DNEE outsources this task to local parcel-delivery service or postal office. Our third and final recommendation for DNEE, should they consider future optimization, is to employ their own delivery trucks fleet out of each distributioin center. We suggest that they use the traveling salesman-problem (TSP) model to cut on the delivery cost to consumers. In such a model, we would assume that trucks start from the distribution center, visit a set of customers in a specific order, and then come back to the distribution center. Hence, the optimization model would handle shipping products to the end consumers from any of distribution center that have a feasible shipping route, while minimizing the total shipping cost (number of trucks per distribution center, gas cost, trip time, number of employees a day or labor hours etc.) 
